In [ ]:
!pip install pytube
!pip install pydub

In [ ]:
from  pytube import YouTube
import os
from pydub import AudioSegment
import csv

def cut_audio(input_file, output_file, start_time, end_time):
    audio = AudioSegment.from_file(input_file)
    audio = audio.set_frame_rate(32000)
    cut_audio = audio[start_time:end_time]
    cut_audio.export(output_file, format="mp3")


def get_audio_clip(video_id, start, end):
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    selected_video = YouTube(video_url)
    audio = selected_video.streams.filter(only_audio = True).first()
    path_dest = audio.download("./download", filename=f"{video_id}.mp3")

    cut_audio(path_dest, path_dest, start*1000, end*1000)


with open('balanced_train_segments.csv', mode ='r')as file:
  csvFile = csv.reader(file)
  for lines in csvFile:
     video_id = lines[0]
     start = lines[1]
     end = lines[2]
     try:
        get_audio_clip(video_id,float(start),float(end))
     except:
        continue



In [18]:

def plot_waveform(waveform, sample_rate):

    waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate
    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle("waveform")
    plt.show()

def get_mixture_audio(audio1,audio2):

    waveform_s1, sample_rate_s1 = torchaudio.load(f"{audio1}.mp3")
    waveform_s2,sample_rate_s2 = torchaudio.load(f"{audio2}.mp3")

    E1 = torch.square(torch.norm(waveform_s1,p=2))
    E2 = torch.square(torch.norm(waveform_s2,p=2))

    alpha = torch.sqrt(E1/E2)

    x = waveform_s1 + alpha * waveform_s2
    return x


In [27]:
import random
import os
import torch
from pydub import AudioSegment
import matplotlib as plt
import math
import torchaudio

def random_sample(mode):

  filenames = random.sample(os.listdir("./download"), 2)
  print(filenames)
  audio1 = filenames[0]
  audio2 = filenames[1]
  audio1 = AudioSegment.from_file(f'./download/{audio1}')
  audio1 = audio1.set_frame_rate(32000)
  audio2 = AudioSegment.from_file(f'./download/{audio2}')
  audio2 = audio2.set_frame_rate(32000)

  if mode == "training":

    duration1 = audio1.duration_seconds
    duration2 = audio2.duration_seconds

    start_time1 = random.uniform(0,(duration1-5))
    start_time2 = random.uniform(0,(duration2-5))

    clipped_audio1 = audio1[start_time1:start_time1+5]
    clipped_audio2 = audio2[start_time2:start_time2+5]

    clipped_audio1.export('./tmp/audio1.mp3', format="mp3")
    clipped_audio2.export('./tmp/audio2.mp3', format="mp3")


    mixed = get_mixture_audio("tmp/audio1","tmp/audio2")

    out = torch.stft(mixed,n_fft=1024,hop_length=512,return_complex=True)
    out_real = out[:, :, :, 0]
    out_imag = out[:, :, :, 1]

    out_abs = torch.sqrt(out_real**2 + out_imag**2)

    plt.pcolormesh(out_abs[0])



    print('training')




random_sample('training')


['-FFx68qSAuY.mp3', '-JBvPwxaVVg.mp3']


RuntimeError: ignored